In [1]:
import itertools
import json
import os

import numpy as np
from pathlib import Path
import pandas as pd

In [2]:
BASE_DIR = f"../../evaluation/output_evals/europarl_st"
DIRECTION_PAIRS = [f'en_{x}' for x in ("de","es","fr","it","nl","pt","zh")]
DIRECTION_PAIRS.extend(f'{x}_en' for x in ("de","fr","it","pt"))
DIRECTION_PAIRS

['en_de',
 'en_es',
 'en_fr',
 'en_it',
 'en_nl',
 'en_pt',
 'en_zh',
 'de_en',
 'fr_en',
 'it_en',
 'pt_en']

In [3]:
SYSTEM_NAMES = os.listdir(BASE_DIR)
SYSTEM_NAMES

['phi4multimodal',
 'tower_owsm4.0-ctc',
 'gemma_owsm4.0-ctc',
 'owsm4.0-ctc',
 'gemma_seamlessm4t',
 'qwen2audio-7b',
 'gemma_whisper',
 'tower_canary-v2',
 'seamlessm4t',
 'desta2-8b',
 'tower_seamlessm4t',
 'voxtral-small-24b',
 'whisper',
 'spirelm',
 'canary-v2',
 'tower_whisper',
 'gemma_canary-v2',
 'aya_canary-v2']

In [8]:
def load_results_summaries(base_dir, direction_pairs, system_names):
    """
    Loads all result summaries from a directory structure.

    Args:
        base_dir (str or Path): The base directory for the evaluation outputs.
        direction_pairs (list): A list of language direction strings (e.g., 'en_de').
        system_names (list): A list of system name strings.

    Returns:
        dict: A nested dictionary containing the loaded data, structured as
              {direction: {system: [results]}}.
    """
    base_path = Path(base_dir)
    all_results = {}

    # Use itertools.product to cleanly iterate over all combinations
    for direction, system in itertools.product(direction_pairs, system_names):
        summary_path = base_path / system / direction / 'results.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with summary_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]
                
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {summary_path}")
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {summary_path}: {e}")
            all_results[direction][system] = None

    return all_results

In [9]:
def convert_results_to_dataframe(results_data):
    """
    Converts the nested dictionary of results into a single pandas DataFrame.

    Each row corresponds to a single entry, with 'direction' and 'system'
    columns added, and all 'metrics' unpacked into separate columns.
    """
    all_records = []
    for direction, systems in results_data.items():
        for system, records in systems.items():
            if records is None:
                continue
            for record in records:
                # Separate metrics from the record
                metrics = record.pop("metrics", {})  # safely get metrics
                # Merge everything into one flat dict
                flat_record = {
                    "direction": direction,
                    "system": system,
                    **record,
                    **metrics,  # unpack metrics into top-level keys
                }
                all_records.append(flat_record)

    if not all_records:
        print("No records were found to create a DataFrame.")
        return pd.DataFrame()

    df = pd.DataFrame(all_records)

    # Put identifying info up front
    original_cols = [c for c in df.columns if c not in ["direction", "system"]]
    df = df[["direction", "system"] + original_cols]

    return df

In [10]:
def compute_strict_scores(df):
    """
    Computes mean metric scores and strict scores grouped by (system, accent).
    
    Expects columns:
      - system
      - accent
      - xcomet_qe_score
      - metricx_qe_score
      - linguapy_score (list/tuple of [flag, lang])
    """
    df = df.copy()

    # --- Split linguapy_score into two separate columns ---
    df[["linguapy_flag", "linguapy_lang"]] = pd.DataFrame(
        df["linguapy_score"].tolist(), index=df.index
    )

    # --- Define penalties ---
    penalty_by_metric = {
        "metricx_qe": 25,
        "xcomet_qe": 0,
    }

    # --- Strict score per row ---
    for metric in penalty_by_metric.keys():
        df[f"{metric}_strict"] = df.apply(
            lambda row: row[f"{metric}_score"]
            if row["linguapy_flag"] == 0
            else penalty_by_metric[metric],
            axis=1,
        )

    # --- Aggregate by system × accent ---
    agg_cols = {
        "linguapy_flag": "mean",  # average from 0–1
    }
    for metric in penalty_by_metric.keys():
        agg_cols[f"{metric}_score"] = "mean"
        agg_cols[f"{metric}_strict"] = "mean"

    result = (
        df.groupby(["system"])
        .agg(agg_cols)
        .reset_index()
        .rename(columns={"linguapy_flag": "linguapy_avg"})
    )

    result['linguapy_avg'] = result['linguapy_avg']*100
    return result

In [11]:
results_full = load_results_summaries(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)

In [14]:
df = convert_results_to_dataframe(results_full)
df.head(3)

,direction,system,dataset_id,sample_id,src_lang,tgt_lang,output,bleu_score,chrf_score,xcomet_score,xcomet_qe_score,metricx_score,metricx_qe_score,linguapy_score
0,en_de,phi4multimodal,europarl_st,0,en,de,"Es ist nur Gier, Euphorie und billiges Geld, d...",36.711085,56.813599,0.855230,0.853524,2.849095,3.059091,"[0, GERMAN]"
1,en_de,phi4multimodal,europarl_st,1,en,de,Was ist mit den Schwächen des regionalen Indus...,5.464223,29.173827,0.839980,0.843512,5.043487,4.982013,"[0, GERMAN]"
2,en_de,phi4multimodal,europarl_st,2,en,de,Was ist mit verzerrten Gehaltsstrukturen ohne ...,9.224861,24.398285,0.989102,0.993245,0.757576,0.635374,"[0, GERMAN]"


In [ ]:
col_map = {
    "linguapy_avg":"LinguaPy",
    "metricx_qe_strict":"QEMetricX_24-Strict-linguapy",
    "xcomet_qe_strict": "XCOMET-QE-Strict-linguapy"
}

#Collapse and get the metrics balanced by the linguapy score
for pair in DIRECTION_PAIRS:
    sub_df = df[df['direction']==pair]
    sub_df = compute_strict_scores(sub_df)
    #Standardize col names
    sub_df = sub_df.rename(columns=col_map)
    #Save 
    sub_df.to_csv(f"europarl_st-{pair}.csv",index=False)